In [ ]:
import itertools as it
import functools as ft
import operator
from fractions import Fraction
import math

class CalkinWilf:
    ROOT_LEVEL = 0
    ROOT_LEVEL_INDEX = 0
    ROOT_MOVE_LIST = []
    ROOT_FRACTION_TUPLE = (1, 1)

    def __init__(self, level=None, level_index=None, move_list=None, fraction_tuple=None):
        if level is None:
            self.level = type(self).ROOT_LEVEL
            self.level_index = type(self).ROOT_LEVEL_INDEX
            self.bfs_index = 2 ** type(self).ROOT_LEVEL + type(self).ROOT_LEVEL_INDEX
            self.move_list = type(self).ROOT_MOVE_LIST
            self.fraction_tuple = type(self).ROOT_FRACTION_TUPLE
        else:
            self.level = level
            self.level_index = level_index
            self.bfs_index = 2 ** self.level + self.level_index
            self.move_list = move_list
            self.fraction_tuple = fraction_tuple
    
    def __repr__(self):
        return f"{type(self).__name__}({self.level}, {self.level_index}, {self.move_list}, {self.fraction_tuple})"
    
    def __eq__(self, other):
        return self.level == other.level and self.level_index == other.level_index
    
    def __lt__(self, other):
        return len(self.move_list) < len(other.move_list) and other.move_list[:len(self.move_list)] == self.move_list
    
    def L(self):
        new_level = self.level + 1
        new_level_index = 2 * self.level_index
        new_move_list = self.move_list + ["L"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a, a + b)
        return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def R(self):
        new_level = self.level + 1
        new_level_index = 2 * self.level_index + 1
        new_move_list = self.move_list + ["R"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a + b, b)
        return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def P(self):
        if self.level == type(self).ROOT_LEVEL:
            return None
        else:
            new_level = self.level - 1
            new_level_index = self.level_index // 2
            move_list = self.move_list
            new_move_list = move_list[0:-1]
            a, b = self.fraction_tuple
            if a < b:
                new_fraction_tuple = (a, b - a)
            else:
                new_fraction_tuple = (a - b, b)
            return type(self)(new_level, new_level_index, new_move_list, new_fraction_tuple)
    
    def run_list(self):
        return [(k, len(list(g))) for k, g in it.groupby(self.move_list)]

    @classmethod
    def find_move_list(cls, fraction_tuple, move_list):
        if fraction_tuple == (1, 1):
            return move_list
        else:
            a, b = fraction_tuple
            if a < b:
                new_move_list = ["L"] + move_list
                new_fraction_tuple = (a, b - a)
            else:
                new_move_list = ["R"] + move_list
                new_fraction_tuple = (a - b, b)
            return cls.find_move_list(new_fraction_tuple, new_move_list)
        
    @classmethod
    def find_node_from_fraction_tuple(cls, fraction_tuple):
        move_list = cls.find_move_list(fraction_tuple, [])
        level = len(move_list)
        level_index = cls.find_level_index(move_list)
        return cls(level, level_index, move_list, fraction_tuple)
    
    @classmethod
    def make_bfs_node_list(cls, depth):
        cw_tree = []
        N = cls()
        level_node_list = []
        level_node_list.append(N)
        cw_tree.append(level_node_list)
        for n in range(depth):
            previous_level_node_list = level_node_list
            level_node_list = []
            for N in previous_level_node_list:
                level_node_list.append(N.L())
                level_node_list.append(N.R())
            cw_tree.append(level_node_list)
        cw_bfs_list = ft.reduce(operator.iadd, cw_tree, [])
        return cw_bfs_list
    
    @classmethod
    def bfs_fraction_tuple_from_bits(cls, n: int):
        _, run_length_list = cls.reversed_binary_string_runs(n)
        convergent_list = cls.find_convergents(run_length_list)
        cf_value = convergent_list[-1]
        return cf_value

    @staticmethod
    def bfs_fraction_tuple_from_recursion(n):
        @ft.lru_cache
        def recurse_q(n):
            if n == 0:
                return Fraction(1, 1)
            else:
                return 1 / (2 * int(recurse_q(n - 1)) - recurse_q(n - 1) + 1)
        q = recurse_q(n - 1)
        return (q.numerator, q.denominator)
    
    @staticmethod
    def reversed_binary_string_runs(n: int):
        binary_string = f"{n:b}"
        runs = [(bit, len(list(g))) for bit, g in it.groupby(reversed(binary_string))]
        move_tuple, run_length_tuple = zip(*runs)
        move_list, run_length_list = list(move_tuple), list(run_length_tuple)
        if move_list[0] == "0":
            run_length_list.insert(0, 0)
            move_list.insert(0, "1")
        return move_list, run_length_list

    @staticmethod
    def find_convergents(a: list[int]) -> list[tuple[int, int]]:
        N = len(a)
        if N == 0:
            return []
        p_prev2, p_prev1 = 0, 1
        q_prev2, q_prev1 = 1, 0
        convergent_list = []
        for k in range(N):
            p_k = a[k] * p_prev1 + p_prev2
            q_k = a[k] * q_prev1 + q_prev2
            convergent_list.append((p_k, q_k))
            p_prev2, p_prev1 = p_prev1, p_k
            q_prev2, q_prev1 = q_prev1, q_k        
        return convergent_list

    @staticmethod
    def find_level_index(move_list: list[str]) -> int:
        move_encode_dict = {"L": "0", "R": "1"}
        move_list_encoded = [move_encode_dict[move] for move in move_list]
        move_list_string = "".join(move_list_encoded)
        level_index = int(move_list_string, base=2)
        return level_index
    
    @staticmethod
    @ft.lru_cache
    def fusc(n):
        def fusc_recurse(n):
            if n == 0:
                return 0
            elif n == 1:
                return 1
            else:
                if n % 2 == 0:
                    return fusc_recurse(n // 2)
                else:
                    m = (n - 1) // 2
                    return fusc_recurse(m) + fusc_recurse(m + 1)
        return fusc_recurse(n)

if __name__ == "__main__":
    n = 102
    depth = int(math.log2(n))

    cw_bfs_list = CalkinWilf.make_bfs_node_list(depth)
    N1 = cw_bfs_list[n]
    assert cw_bfs_list[n].bfs_index - 1 == n

    N2 = N1.R()
    assert N2 > N1

    assert CalkinWilf.bfs_fraction_tuple_from_bits(n + 1) == N1.fraction_tuple

    fusc_fraction_tuple = (CalkinWilf.fusc(n + 1), CalkinWilf.fusc(n + 2))
    bfs_fraction_tuple = CalkinWilf.bfs_fraction_tuple_from_bits(n + 1)
    assert fusc_fraction_tuple == bfs_fraction_tuple

    bfs_fraction_tuple = CalkinWilf.bfs_fraction_tuple_from_bits(n + 1)
    bfs_fraction_tuple_from_recursion = CalkinWilf.bfs_fraction_tuple_from_recursion(n + 1)
    assert bfs_fraction_tuple, bfs_fraction_tuple_from_recursion


CalkinWilf(6, 39, ['R', 'L', 'L', 'R', 'R', 'R'], (17, 5))
